In [ ]:
from pathlib import Path
import shutil

# Kaggle の入力データセット slug（username/slug の slug 部分）をセット
DATASET_SLUG = 'gns-solid-api'  # 変更する場合はここを更新
DATASET_ROOT = Path(f'/kaggle/input/{DATASET_SLUG}')
CODE_ZIP = DATASET_ROOT / 'code.zip'
WORK_ROOT = Path('/kaggle/working')
repo_dir = WORK_ROOT / 'code'

# 既存の code ディレクトリをクリアして zip を展開
if repo_dir.exists():
    shutil.rmtree(repo_dir)
repo_dir.mkdir(parents=True, exist_ok=True)

if not CODE_ZIP.exists():
    raise FileNotFoundError(f'{CODE_ZIP} が見つかりません。Add Data で {DATASET_SLUG} を追加してください。')

!unzip -q "$CODE_ZIP" -d "$WORK_ROOT"
%cd $repo_dir
# 依存パッケージをインストール (pyproject.toml に基づく)
!pip install -q $repo_dir


In [ ]:
!pip install torch-geometric 

In [ ]:
from pathlib import Path
import yaml

REPO = Path('/kaggle/working/code')
cfg_path = REPO / 'config.yaml'

with cfg_path.open('r', encoding='utf-8') as f:
    cfg = yaml.safe_load(f)

cfg['method'] = 'hamiltonian_sph'
cfg['amp_enable'] = False
cfg['ntraining_steps'] = 100
cfg['train_dataset_count'] = 10
cfg['valid_dataset_count'] = 2
cfg['validation_interval'] = 10

# Kaggle 入力データセットのパスに合わせて上書き
dataset_root = '/kaggle/input/dam-break-left/dam_break_left'
cfg['data_path'] = dataset_root
cfg.setdefault('scenario_options', {}).setdefault('fluid', {})['dataset'] = dataset_root

with cfg_path.open('w', encoding='utf-8') as f:
    yaml.safe_dump(cfg, f, allow_unicode=True)


In [ ]:
%cd /kaggle/working/code
# GPUを2枚使う設定
!torchrun --nproc_per_node=2 src/train.py --config config.yaml


In [ ]:
from pathlib import Path
import yaml

REPO = Path('/kaggle/working/code')
cfg_path = REPO / 'config_rollout.yaml'

with cfg_path.open('r', encoding='utf-8') as f:
    cfg = yaml.safe_load(f)

cfg['method'] = 'hamiltonian_sph'
cfg['rollout_inference_max_examples'] = 1

# 推論時のデータセットパスを上書き
dataset_root = '/kaggle/input/dam-break-left/dam_break_left'
cfg.setdefault('scenario_options', {}).setdefault('fluid', {})['dataset'] = dataset_root

with cfg_path.open('w', encoding='utf-8') as f:
    yaml.safe_dump(cfg, f, allow_unicode=True)


In [ ]:
%cd /kaggle/working/code
!python src/train.py --config config_rollout.yaml
!python analyze_rollouts.py
!python visualize_rollout.py
